# How similar is the text?

In [1]:
import spacy

import gensim.downloader as api

In [2]:
load_nlp_model = spacy.load("en_core_web_md")
pretrained_word_two_vec_model = api.load('word2vec-google-news-300')

In [3]:
prediction_txt = "I predict it will rain tomorrow."
observation_txt = "It rained tomorrow."

prediction_txt = "Apple's stock will rise by 10%."
observation_txt = "Apple's stock rose by 11%."
observation_txt_2 = "Apple's stock rose by 30%."

In [4]:
prediction = load_nlp_model(prediction_txt)
observation = load_nlp_model(observation_txt)
observation_2 = load_nlp_model(observation_txt_2)

p_o = prediction.similarity(observation)
p_o_2 = prediction.similarity(observation_2)

o_p = observation.similarity(prediction)
o_p_2 = observation_2.similarity(prediction)

In [5]:
p_o, p_o_2

(0.9399013747850621, 0.9399013747850621)

In [6]:
o_p, o_p_2

(0.9399013747850621, 0.9399013747850621)

In [7]:
wmdistance_score = pretrained_word_two_vec_model.wmdistance(prediction_txt, observation_txt)
wmdistance_score

0.25614710912925986

In [8]:
sentence_obama = 'Obama speaks to the media in Illinois'.lower().split()
sentence_president = 'The president greets the press in Chicago'.lower().split()

similarity = pretrained_word_two_vec_model.wmdistance(sentence_obama, sentence_president)
print(f"{similarity:.4f}")

prediction_txt_transformed = prediction_txt.lower().split()
observation_txt_transformed = observation_txt.lower().split()
wmdistance_score = pretrained_word_two_vec_model.wmdistance(prediction_txt_transformed, observation_txt_transformed)
wmdistance_score

0.7331


0.564630188695818